In [2]:
import pandas as pd
import re
from collections import Counter

In [6]:
df_2021_emp = pd.read_excel('AY 2021 BBE EFR Free Response.xlsx',sheet_name=0)  
df_2021_stud = pd.read_excel('AY 2021 BBE ES&P Free Response.xlsx',sheet_name=0)  

In [10]:
df_2021_emp.rename(columns={ df_2021_emp.columns[8]: "Emp_Comments" }, inplace = True)
df_2021_emp

,Responder ID,Work Terms,College,Major,Concentration,Class BOT,Citizenship Status,Co-op #,Emp_Comments
0,3581,Fall/Winter,B,ACCT,5COP,SR,Domestic,Second,Ability to get along with and work well with o...
1,3584,Fall/Winter,B,INT2,5COP,SR,Domestic,First,Will is very good at tackling problems head on...
2,3588,Fall/Winter,B,FIN,5COP,SR,Domestic,Third/Final,"Proactive, focused, detail-oriented, efficient"
3,42,Spring/Summer,B,FIN,5COP,PJ,Domestic,First,She is a very fast learner and is always looki...
4,3600,Fall/Winter,B,MKTG,5COP,SR,Domestic,Third/Final,John wants to learn and improve. I was impress...
...,...,...,...,...,...,...,...,...,...
666,5773,Fall/Winter,B,FIN,4COP,SR,Domestic,First,Very hardworking and always ready to learn more.
667,3541,Spring/Summer,BE,ECON,5COP,JR,International,First,Damaris was strong and consistent throughout h...
668,3548,Spring/Summer,B,FIN,5COP,SO,Domestic,First,Drue is very personable and professional and d...
669,3550,Spring/Summer,B,LGST,5COP,JR,Domestic,First,Her abilities to analyze and interpret data ar...


In [13]:
df_2021_stud.rename(columns={ df_2021_stud.columns[8]: "Stud_Comments" }, inplace = True)
df_2021_stud

,Responder ID,Work Terms,College,Major,Conc,Class,Citizenship Status,Co-op #,Stud_Comments
0,3572,Fall/Winter,B,FIN,4COP,SR,International,First/Only,this co-op really helped me gain leadership qu...
1,3574,Fall/Winter,B,LGST,4COP,SR,Domestic,First/Only,I really wanted to build my network and begin ...
2,3581,Fall/Winter,B,ACCT,5COP,SR,Domestic,Second,Although I do not see myself working in financ...
3,3583,Fall/Winter,B,FIN,5COP,JR,Domestic,Second,At my previous co-op I was told I need to spea...
4,3584,Fall/Winter,B,INT2,5COP,SR,Domestic,First,I never expected to have a job in forensic acc...
...,...,...,...,...,...,...,...,...,...
1337,3548,Spring/Summer,B,FIN,5COP,SO,Domestic,First,I eventually would like to work at a fund as a...
1338,3550,Spring/Summer,B,LGST,5COP,JR,Domestic,First,It allowed me to look more into the subject of...
1339,3555,Spring,B,FIN,5COP,JR,Domestic,First,NaN
1340,3563,Spring/Summer,BE,ECON,5COP,JR,Domestic,First,This is a great question because I feel like m...


In [16]:
df_2021_stud = df_2021_stud[['Responder ID','Stud_Comments']]
df_2021_emp = df_2021_emp[['Responder ID','Emp_Comments']]

In [17]:
df = pd.merge(df_2021_stud,df_2021_emp,how='left',on='Responder ID')
df

,Responder ID,Stud_Comments,Emp_Comments
0,3572,this co-op really helped me gain leadership qu...,NaN
1,3574,I really wanted to build my network and begin ...,NaN
2,3581,Although I do not see myself working in financ...,Ability to get along with and work well with o...
3,3583,At my previous co-op I was told I need to spea...,NaN
4,3584,I never expected to have a job in forensic acc...,Will is very good at tackling problems head on...
...,...,...,...
1339,3548,I eventually would like to work at a fund as a...,Drue is very personable and professional and d...
1340,3550,It allowed me to look more into the subject of...,Her abilities to analyze and interpret data ar...
1341,3555,NaN,NaN
1342,3563,This is a great question because I feel like m...,"Zach is very professional, communicative and p..."


# Pre-processing

In [18]:
df['Stud_Clean_Text'] = df['Stud_Comments'].str.lower()
df['Emp_Clean_Text'] = df['Emp_Comments'].str.lower()
df.head(2)

,Responder ID,Stud_Comments,Emp_Comments,Stud_Clean_Text,Emp_Clean_Text
0,3572,this co-op really helped me gain leadership qu...,NaN,this co-op really helped me gain leadership qu...,NaN
1,3574,I really wanted to build my network and begin ...,NaN,i really wanted to build my network and begin ...,NaN


# Removing Punctuation

In [19]:
import string
string.punctuation
def remove_punctuations(text):
    punctuations = string.punctuation
    return text.translate(str.maketrans('','',punctuations)) 

In [20]:
df = df.fillna(" ")
df['Stud_Clean_Text'] = df['Stud_Clean_Text'].apply(lambda x : remove_punctuations(x))
df['Emp_Clean_Text'] = df['Emp_Clean_Text'].apply(lambda x : remove_punctuations(x))
df.head(2)

,Responder ID,Stud_Comments,Emp_Comments,Stud_Clean_Text,Emp_Clean_Text
0,3572,this co-op really helped me gain leadership qu...,,this coop really helped me gain leadership qua...,
1,3574,I really wanted to build my network and begin ...,,i really wanted to build my network and begin ...,


# Removing Stopwords

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ananya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join(word for word in text.split() if word not in STOPWORDS)
df['Stud_Clean_Text'] = df['Stud_Clean_Text'].apply(lambda x : remove_stopwords(x))
df['Emp_Clean_Text'] = df['Emp_Clean_Text'].apply(lambda x : remove_stopwords(x))
df.head(5)

,Responder ID,Stud_Comments,Emp_Comments,Stud_Clean_Text,Emp_Clean_Text,Stud_Sentiment,Emp_Sentiment,Stud_Sentiment_Class,Emp_Sentiment_Class,Flag
0,3572,this co-op really helped me gain leadership qu...,,coop really helped gain leadership qualities t...,,0.240000,0.000,positive,neutral,Not Aligned
1,3574,I really wanted to build my network and begin ...,,really wanted build network begin solid career...,,0.135105,0.000,positive,neutral,Not Aligned
2,3581,Although I do not see myself working in financ...,Ability to get along with and work well with o...,although see working finance digital future ca...,ability get along work well others regardless ...,0.100658,0.375,positive,positive,Aligned
3,3583,At my previous co-op I was told I need to spea...,,previous coop told need speak learn take charg...,,0.076852,0.000,positive,neutral,Not Aligned
4,3584,I never expected to have a job in forensic acc...,Will is very good at tackling problems head on...,never expected job forensic accountingfinancia...,good tackling problems head even though coop s...,0.066802,0.500,positive,positive,Aligned


# Sentiment Analysis

In [33]:
!pip install textblob
from textblob import TextBlob
def analyze_sentiment(text):
    if pd.isna(text):
        return 0
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [30]:
df['Stud_Sentiment'] = df['Stud_Clean_Text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df['Emp_Sentiment'] = df['Emp_Clean_Text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)


def classify_sentiment(polarity):
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

df['Stud_Sentiment_Class'] = df['Stud_Sentiment'].apply(classify_sentiment)
df['Emp_Sentiment_Class'] = df['Emp_Sentiment'].apply(classify_sentiment)


# output_file_path = "output_sentiment_analysis.xlsx"
# df.to_excel(output_file_path, index=False, engine='openpyxl')
df

,Responder ID,Stud_Comments,Emp_Comments,Stud_Clean_Text,Emp_Clean_Text,Stud_Sentiment,Emp_Sentiment,Stud_Sentiment_Class,Emp_Sentiment_Class
0,3572,this co-op really helped me gain leadership qu...,,coop really helped gain leadership qualities t...,,0.240000,0.000,positive,neutral
1,3574,I really wanted to build my network and begin ...,,really wanted build network begin solid career...,,0.135105,0.000,positive,neutral
2,3581,Although I do not see myself working in financ...,Ability to get along with and work well with o...,although see working finance digital future ca...,ability get along work well others regardless ...,0.100658,0.375,positive,positive
3,3583,At my previous co-op I was told I need to spea...,,previous coop told need speak learn take charg...,,0.076852,0.000,positive,neutral
4,3584,I never expected to have a job in forensic acc...,Will is very good at tackling problems head on...,never expected job forensic accountingfinancia...,good tackling problems head even though coop s...,0.066802,0.500,positive,positive
...,...,...,...,...,...,...,...,...,...
1339,3548,I eventually would like to work at a fund as a...,Drue is very personable and professional and d...,eventually would like work fund analyst eventu...,drue personable professional great job communi...,0.200000,0.420,positive,positive
1340,3550,It allowed me to look more into the subject of...,Her abilities to analyze and interpret data ar...,allowed look subject nonprofit law laws also t...,abilities analyze interpret data among strengt...,0.016667,0.000,positive,neutral
1341,3555,,,,,0.000000,0.000,neutral,neutral
1342,3563,This is a great question because I feel like m...,"Zach is very professional, communicative and p...",great question feel like personal academic pro...,zach professional communicative proactive curi...,0.109524,0.000,positive,neutral


In [36]:
def match(df):
    if (df['Stud_Sentiment_Class'] == 'positive') & (df['Emp_Sentiment_Class'] == 'positive'):
        return 'Aligned'
    elif (df['Stud_Sentiment_Class'] == 'neutral') & (df['Emp_Sentiment_Class'] == 'neutral'):
        return 'Aligned'   
    else:
        return 'Not Aligned'
    
df['Flag'] = df.apply(match,axis=1)
df

,Responder ID,Stud_Comments,Emp_Comments,Stud_Clean_Text,Emp_Clean_Text,Stud_Sentiment,Emp_Sentiment,Stud_Sentiment_Class,Emp_Sentiment_Class,Flag
0,3572,this co-op really helped me gain leadership qu...,,coop really helped gain leadership qualities t...,,0.240000,0.000,positive,neutral,Not Aligned
1,3574,I really wanted to build my network and begin ...,,really wanted build network begin solid career...,,0.135105,0.000,positive,neutral,Not Aligned
2,3581,Although I do not see myself working in financ...,Ability to get along with and work well with o...,although see working finance digital future ca...,ability get along work well others regardless ...,0.100658,0.375,positive,positive,Aligned
3,3583,At my previous co-op I was told I need to spea...,,previous coop told need speak learn take charg...,,0.076852,0.000,positive,neutral,Not Aligned
4,3584,I never expected to have a job in forensic acc...,Will is very good at tackling problems head on...,never expected job forensic accountingfinancia...,good tackling problems head even though coop s...,0.066802,0.500,positive,positive,Aligned
...,...,...,...,...,...,...,...,...,...,...
1339,3548,I eventually would like to work at a fund as a...,Drue is very personable and professional and d...,eventually would like work fund analyst eventu...,drue personable professional great job communi...,0.200000,0.420,positive,positive,Aligned
1340,3550,It allowed me to look more into the subject of...,Her abilities to analyze and interpret data ar...,allowed look subject nonprofit law laws also t...,abilities analyze interpret data among strengt...,0.016667,0.000,positive,neutral,Not Aligned
1341,3555,,,,,0.000000,0.000,neutral,neutral,Aligned
1342,3563,This is a great question because I feel like m...,"Zach is very professional, communicative and p...",great question feel like personal academic pro...,zach professional communicative proactive curi...,0.109524,0.000,positive,neutral,Not Aligned


In [37]:
final = pd.pivot_table(df,index='Flag',aggfunc={'Responder ID':pd.Series.nunique})
final

,Responder ID
Flag,
Aligned,566
Not Aligned,775
